In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT))

print("Project root:", PROJECT_ROOT)


Project root: /shared/crollo/faers_signals


In [2]:
import pandas as pd
import duckdb

from faers_signals.config import WAREHOUSE_DB_PATH
from faers_signals.signals_classical import (
    compute_signals_for_drug,
    DisproportionalityConfig,
)

con = duckdb.connect(str(WAREHOUSE_DB_PATH), read_only=True)
con

In [3]:
cfg = DisproportionalityConfig(
    roles=("PS", "SS"),   # only primary and secondary suspect drugs
    min_n11=5,            # slightly stricter than default
    min_total_drug_reports=100
)

signals_atorva = compute_signals_for_drug(
    "ATORVASTATIN",
    cfg=cfg,
    con=con
)

signals_atorva.head(20)


,pt,n11,total_event_reports,total_drug_reports,total_reports,n10,n01,n00,ror,ror_ci_low,ror_ci_high,prr
0,HMG CoA reductase antibody test,12.0,12,28115,9884211,28103.0,0.0,9856096.0,8767.677069,519.076219,148094.168699,8763.779521
1,Endoscopic swallowing evaluation,5.0,5,28115,9884211,28110.0,0.0,9856096.0,3856.817257,213.249329,69754.214061,3856.062989
2,Genital infection viral,13.0,15,28115,9884211,28102.0,2.0,9856094.0,1893.885252,491.182312,7302.382972,1892.976376
3,Clang associations,7.0,8,28115,9884211,28108.0,1.0,9856095.0,1753.223313,303.716555,10120.594146,1752.755904
4,Leukocyte adhesion deficiency,10.0,13,28115,9884211,28105.0,3.0,9856093.0,1052.046059,313.781617,3527.296849,1051.653542
5,Acute necrotising myelitis,7.0,9,28115,9884211,28108.0,2.0,9856094.0,1051.933881,251.377564,4402.003397,1051.653542
6,Fibromatosis colli of infancy,9.0,12,28115,9884211,28106.0,3.0,9856093.0,951.817330,279.433603,3242.116264,951.496062
7,Pregnenolone deficiency,11.0,15,28115,9884211,28104.0,4.0,9856092.0,896.219180,301.354339,2665.330195,895.853018
8,Drug dechallenge positive,6.0,10,28115,9884211,28109.0,4.0,9856092.0,506.468562,152.238250,1684.927443,506.351706
9,Lecithin-cholesterol acyltransferase deficiency,6.0,10,28115,9884211,28109.0,4.0,9856092.0,506.468562,152.238250,1684.927443,506.351706


In [4]:
strong = signals_atorva[
    (signals_atorva["n11"] >= 10) &
    (signals_atorva["ror"] >= 2.0) &
    (signals_atorva["ror_ci_low"] > 1.0)
].copy()

strong.head(20)


,pt,n11,total_event_reports,total_drug_reports,total_reports,n10,n01,n00,ror,ror_ci_low,ror_ci_high,prr
0,HMG CoA reductase antibody test,12.0,12,28115,9884211,28103.0,0.0,9856096.0,8767.677069,519.076219,148094.168699,8763.779521
2,Genital infection viral,13.0,15,28115,9884211,28102.0,2.0,9856094.0,1893.885252,491.182312,7302.382972,1892.976376
4,Leukocyte adhesion deficiency,10.0,13,28115,9884211,28105.0,3.0,9856093.0,1052.046059,313.781617,3527.296849,1051.653542
7,Pregnenolone deficiency,11.0,15,28115,9884211,28104.0,4.0,9856092.0,896.219180,301.354339,2665.330195,895.853018
10,Muscle relaxant therapy,10.0,17,28115,9884211,28105.0,7.0,9856089.0,490.954628,192.331289,1253.235748,490.771653
13,Lipoprotein (a) decreased,12.0,22,28115,9884211,28103.0,10.0,9856086.0,417.508008,183.779337,948.490402,417.322834
15,Non-high-density lipoprotein cholesterol decre...,14.0,26,28115,9884211,28101.0,12.0,9856084.0,406.848674,190.922597,866.978797,406.639370
17,"Hallucination, gustatory",10.0,19,28115,9884211,28105.0,9.0,9856087.0,387.595680,161.135926,932.321026,387.451305
18,Degenerative mitral valve disease,20.0,39,28115,9884211,28095.0,19.0,9856077.0,368.796333,198.369960,685.641793,368.528164
22,Cystostomy,12.0,27,28115,9884211,28103.0,15.0,9856081.0,282.827862,134.242389,595.874376,282.702565


In [5]:
strong.sort_values("n11", ascending=False).head(20)

,pt,n11,total_event_reports,total_drug_reports,total_reports,n10,n01,n00,ror,ror_ci_low,ror_ci_high,prr
288,Myalgia,2333.0,66796,28115,9884211,25782.0,64463.0,9791633.0,13.747510,13.167937,14.352592,12.689525
115,Rhabdomyolysis,1506.0,15524,28115,9884211,26609.0,14018.0,9842078.0,39.748221,37.640412,41.974065,37.672030
1138,Arthralgia,1253.0,205310,28115,9884211,26862.0,204057.0,9652039.0,2.207214,2.085326,2.336227,2.153393
738,Acute kidney injury,1109.0,98495,28115,9884211,27006.0,97386.0,9758710.0,4.116733,3.875558,4.372915,3.993742
588,Drug interaction,1036.0,67961,28115,9884211,27079.0,66925.0,9789171.0,5.598646,5.259452,5.959714,5.429116
1133,Asthenia,1000.0,161768,28115,9884211,27115.0,160768.0,9695328.0,2.225159,2.088692,2.370542,2.181562
510,Muscular weakness,859.0,46208,28115,9884211,27256.0,45349.0,9810747.0,6.821901,6.370090,7.305757,6.643926
1106,Pain in extremity,797.0,122166,28115,9884211,27318.0,121369.0,9734727.0,2.341467,2.181777,2.512845,2.303417
27,Immune-mediated myositis,767.0,2102,28115,9884211,27348.0,1335.0,9854761.0,207.084264,189.342229,226.488790,201.458653
813,Muscle spasms,763.0,73565,28115,9884211,27352.0,72802.0,9783294.0,3.751033,3.489456,4.032218,3.676327


In [6]:
top_pt = signals_atorva.sort_values("n11", ascending=False).iloc[0]["pt"]
top_pt

'Myalgia'

In [7]:
drug_norm = "ATORVASTATIN"
pt_norm = top_pt

# Quarterly joint counts (N11 per quarter)
joint_q = con.execute("""
    SELECT
        d.year,
        d.quarter,
        COUNT(DISTINCT r.primaryid) AS n11
    FROM reac_dedup r
    JOIN drug_dedup d
      ON r.primaryid = d.primaryid
    WHERE UPPER(TRIM(d.drugname)) = ?
      AND r.pt = ?
      AND d.year >= 2019
    GROUP BY d.year, d.quarter
    ORDER BY d.year, d.quarter
""", [drug_norm, pt_norm]).fetchdf()

joint_q


,year,quarter,n11
0,2019,Q1,93
1,2019,Q2,130
2,2019,Q3,167
3,2019,Q4,110
4,2020,Q1,121
5,2020,Q2,124
6,2020,Q3,99
7,2020,Q4,95
8,2021,Q1,69
9,2021,Q2,84


In [8]:
import duckdb
from faers_signals.config import WAREHOUSE_DB_PATH

con = duckdb.connect(str(WAREHOUSE_DB_PATH), read_only=True)
con.execute("""
    SELECT drugname, drugname_norm
    FROM drug_dedup_norm
    USING SAMPLE 20
""").fetchdf()

,drugname,drugname_norm
0,PARACETAMOL,PARACETAMOL
1,COENZYME Q10,COENZYME Q10
2,RANITIDINE.,RANITIDINE
3,PROACTIV MD ADAPALENE ACNE TREATMENT,PROACTIV MD ADAPALENE ACNE TREATMENT
4,PROACTIV MD ADAPALENE ACNE TREATMENT,PROACTIV MD ADAPALENE ACNE TREATMENT
5,PROACTIV MD DAILY OIL CONTROL SPF 30,PROACTIV MD DAILY OIL CONTROL SPF 30
6,Proactiv MD Balancing Toner,PROACTIV MD BALANCING TONER
7,ZANTAC,ZANTAC
8,LAMICTAL,LAMICTAL
9,CLOZAPINE,CLOZAPINE


In [9]:
from faers_signals.signals_classical import compute_signals_for_drug, DisproportionalityConfig

cfg = DisproportionalityConfig(
    roles=("PS", "SS"),
    min_n11=5,
    min_total_drug_reports=100
)

signals = compute_signals_for_drug("atorvastatin", cfg=cfg)
signals.head(10)


,pt,n11,total_event_reports,total_drug_reports,total_reports,n10,n01,n00,ror,ror_ci_low,ror_ci_high,prr
0,HMG CoA reductase antibody test,12.0,12,28115,9884211,28103.0,0.0,9856096.0,8767.677069,519.076219,148094.168699,8763.779521
1,Endoscopic swallowing evaluation,5.0,5,28115,9884211,28110.0,0.0,9856096.0,3856.817257,213.249329,69754.214061,3856.062989
2,Genital infection viral,13.0,15,28115,9884211,28102.0,2.0,9856094.0,1893.885252,491.182312,7302.382972,1892.976376
3,Clang associations,7.0,8,28115,9884211,28108.0,1.0,9856095.0,1753.223313,303.716555,10120.594146,1752.755904
4,Leukocyte adhesion deficiency,10.0,13,28115,9884211,28105.0,3.0,9856093.0,1052.046059,313.781617,3527.296849,1051.653542
5,Acute necrotising myelitis,7.0,9,28115,9884211,28108.0,2.0,9856094.0,1051.933881,251.377564,4402.003397,1051.653542
6,Fibromatosis colli of infancy,9.0,12,28115,9884211,28106.0,3.0,9856093.0,951.817330,279.433603,3242.116264,951.496062
7,Pregnenolone deficiency,11.0,15,28115,9884211,28104.0,4.0,9856092.0,896.219180,301.354339,2665.330195,895.853018
8,Drug dechallenge positive,6.0,10,28115,9884211,28109.0,4.0,9856092.0,506.468562,152.238250,1684.927443,506.351706
9,Lecithin-cholesterol acyltransferase deficiency,6.0,10,28115,9884211,28109.0,4.0,9856092.0,506.468562,152.238250,1684.927443,506.351706


In [10]:
import duckdb
from faers_signals.config import WAREHOUSE_DB_PATH

con = duckdb.connect(str(WAREHOUSE_DB_PATH), read_only=True)

con.execute("""
    SELECT drugname_norm, COUNT(*) AS n
    FROM drug_dedup_norm
    GROUP BY drugname_norm
    ORDER BY n DESC
    LIMIT 30
""").fetchdf()


,drugname_norm,n
0,ZANTAC,921746
1,RANITIDINE,550083
2,DUPIXENT,471720
3,PREDNISONE,410931
4,REVLIMID,387385
5,HUMIRA,373171
6,METHOTREXATE,358164
7,MOUNJARO,278949
8,DEXAMETHASONE,230677
9,INFLECTRA,222671


In [12]:
import duckdb
from faers_signals.config import WAREHOUSE_DB_PATH

con = duckdb.connect(str(WAREHOUSE_DB_PATH), read_only=True)

# Does the table exist?
con.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
      AND table_name = 'signals_quarterly'
""").fetchdf()


,table_name


In [11]:
import duckdb
from faers_signals.config import WAREHOUSE_DB_PATH

con = duckdb.connect(str(WAREHOUSE_DB_PATH), read_only=True)

con.execute("SELECT COUNT(*) FROM signals_quarterly").fetchall()
con.execute("DESCRIBE signals_quarterly").fetchdf().head()
con.execute("""
    SELECT year, quarter, COUNT(*) AS n_pairs
    FROM signals_quarterly
    GROUP BY year, quarter
    ORDER BY year, quarter
""").fetchdf()


CatalogException: Catalog Error: Table with name signals_quarterly does not exist!
Did you mean "sqlite_master"?

LINE 1: SELECT COUNT(*) FROM signals_quarterly
                             ^

In [ ]:
import duckdb
from faers_signals.config import WAREHOUSE_DB_PATH

con = duckdb.connect(str(WAREHOUSE_DB_PATH), read_only=True)

con.execute("SELECT COUNT(*) FROM signals_quarterly").fetchall()
con.execute("SELECT * FROM signals_quarterly LIMIT 5").fetchdf()
